## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

In [15]:
import pandas as pd
import numpy as np

In [16]:
tracks_list = []
with open('train') as f:
    lines = f.readlines()

In [17]:
data = ((user_id, track, order) for user_id, line in enumerate(lines) for order, track in enumerate(line.strip().split(' ')[::-1]))

In [18]:
df = pd.DataFrame(data, columns=['user_id', 'track', 'order'])

In [19]:
df

,user_id,track,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4
...,...,...,...
94188629,1160083,19120,251
94188630,1160083,326821,252
94188631,1160083,214132,253
94188632,1160083,352098,254


In [20]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed=23):
        self.n_folds = n_folds
        self.random_seed = random_seed
        self.p = p
    
    def split(self, df):
        users = df["user_id"].unique()
        users_count = len(users)
        orders = df["order"]
        orders_mask = orders <= self.p
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df["user_id"].isin(test_fold_users)
            train_mask = ~test_mask
            test_mask = test_mask * orders_mask
            
            yield train_mask, test_mask

In [24]:
cv = UsersKFoldPOut(n_folds=3, p=3)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
    train = df[train_mask]
    test = df[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 62769950, Test: 1546780
Fold#1 | Train: 62769950, Test: 1546780
Fold#2 | Train: 62770016, Test: 1546776


In [25]:
train

,user_id,track,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4
...,...,...,...
94188629,1160083,19120,251
94188630,1160083,326821,252
94188631,1160083,214132,253
94188632,1160083,352098,254


In [26]:
test

,user_id,track,order
54,1,457396,0
55,1,58492,1
56,1,20988,2
57,1,388187,3
1052,12,22082,0
...,...,...,...
94188068,1160080,248232,3
94188326,1160082,281987,0
94188327,1160082,287708,1
94188328,1160082,7045,2
